Objective:
* Go through the list of the target corporate's partners
* Make every combination of two partners
* Google search the combination
* Determine if both partners in one combination appear on the same page

Mainly utilized library:
* google search through python: https://pypi.org/project/googlesearch-python/
* python ip rotator: https://pypi.org/project/requests-ip-rotator/
* AWS web service: https://aws.amazon.com/console/

# Load Libraries

In [1]:
# Python google search package
# !pip install googlesearch-python

# Install requests
!pip install requests

# Install requests-ip-rotator
!pip3 install requests-ip-rotator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.3 MB/s eta 0:00:00


In [2]:
# Regular Python Data analyze library
import pandas as pd
import random

# Use to create iterative combinations
# import itertools

# Google search library
# reference: https://pypi.org/project/googlesearch-python/
# We have copied the full code in
# from googlesearch import search

# Web Request
import requests

# Progress Bar
from tqdm import tqdm

# Requests-ip-rotator to rotate IP
# Aim to avoid 429 HTTP error
# Reference: https://github.com/Ge0rg3/requests-ip-rotator
from requests_ip_rotator import ApiGateway

# Add request error handler
from requests.exceptions import HTTPError, ReadTimeout

# Import Math
import math

# Import Regular Expression
import re

# Prepare Data

Load in the CSV file containing information about the target corporate's sponsors

In [ ]:
# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
df_pool = pd.read_csv('')

# Apply .strip() to all string values in the DataFrame using apply with a lambda function
df_pool = df_pool.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# Now all string values in df_pool will have leading/trailing spaces removed
print("Stripped all leading/trailing spaces from string attributes.")

Stripped all leading/trailing spaces from string attributes.


In [ ]:
# Check dataframe
df_pool

Now we need to do a filtering, if we will only be focusing on a specific aspect of the dataset.\
For example, the sponsors in state "MN" only.

In [5]:
# Filter the df based on specific needs
# df_target = df_pool[df_pool['State'] == 'MN']
# df_target = df_pool[df_pool['Type'] == 'npo']

# Reset the index of the filtered DataFrame and drop the old index
# df_target = df_target.reset_index(drop=True)

# If wanting to examine whole data pool, use this line and comment out all previous lines in this cell
df_target = df_pool

In [6]:
# We dont need any duplications
df_target = df_target.drop_duplicates(subset='Sponsors')
df_target.reset_index(drop=True, inplace=True)

In [ ]:
# Check target dataframe
df_target

# Make Combinations of corporate partners

In [8]:
# Make a list of selected sponsors
# Make sure they are unique
Sponsor_list = df_target['Sponsors'].tolist()
print("Unique number of Sponsors:", len(Sponsor_list))

Unique number of Sponsors: 14


In [9]:
# We can use a small portion of the Sponsor list for testing purposes
# Sponsor_list = Sponsor_list[:30]
# Sponsor_list

In [10]:
# Use iteration tool to create every combination of two partners
# Sponsor_combinations = list(itertools.combinations(Sponsor_list, 2))

# Maually create every combination of two partners
Sponsor_combinations = []
for i in range(len(Sponsor_list)):
  for j in range(i + 1, len(Sponsor_list)):
    Sponsor_combinations.append((Sponsor_list[i], Sponsor_list[j]))

In [11]:
# Check if the list is successfully created
print("Sponsor list length:", len(Sponsor_list))
print("First Combination:", Sponsor_combinations[0])
print("Last Combination:", Sponsor_combinations[len(Sponsor_combinations) - 1])
print("Total number of Combinations:", len(Sponsor_combinations))

Sponsor list length: 14
First Combination: ('Access to Capital for Entrepreneurs', 'Advanced System Development')
Last Combination: ('University of Georgia Small Business Development Center', 'bip ventures')
Total number of Combinations: 91


# Modify code from original python library: googlesearch-python

original website: https://pypi.org/project/googlesearch-python/#description \
original code: https://github.com/Nv7-GitHub/googlesearch

In [12]:
import random

def get_useragent():
    """
    Generates a random user agent string mimicking the format of various software versions.

    The user agent string is composed of:
    - Lynx version: Lynx/x.y.z where x is 2-3, y is 8-9, and z is 0-2
    - libwww version: libwww-FM/x.y where x is 2-3 and y is 13-15
    - SSL-MM version: SSL-MM/x.y where x is 1-2 and y is 3-5
    - OpenSSL version: OpenSSL/x.y.z where x is 1-3, y is 0-4, and z is 0-9

    Returns:
        str: A randomly generated user agent string.
    """
    lynx_version = f"Lynx/{random.randint(2, 3)}.{random.randint(8, 9)}.{random.randint(0, 2)}"
    libwww_version = f"libwww-FM/{random.randint(2, 3)}.{random.randint(13, 15)}"
    ssl_mm_version = f"SSL-MM/{random.randint(1, 2)}.{random.randint(3, 5)}"
    openssl_version = f"OpenSSL/{random.randint(1, 3)}.{random.randint(0, 4)}.{random.randint(0, 9)}"
    return f"{lynx_version} {libwww_version} {ssl_mm_version} {openssl_version}"

In [13]:
"""googlesearch is a Python library for searching Google, easily."""
from time import sleep
from bs4 import BeautifulSoup
from requests import get
from urllib.parse import unquote # to decode the url


def _req(term, results, lang, start, proxies, timeout, safe, ssl_verify, region):
    resp = session.get(
        url="https://www.google.com/search",
        headers={
            "User-Agent": get_useragent(),
            "Accept": "*/*"
        },
        params={
            "q": term,
            "num": results + 2,  # Prevents multiple requests
            "hl": lang,
            "start": start,
            "safe": safe,
            "gl": region,
        },
        proxies=proxies,
        timeout=timeout,
        verify=ssl_verify,
        cookies = {
            'CONSENT': 'PENDING+987', # Bypasses the consent page
            'SOCS': 'CAESHAgBEhIaAB',
        }
    )
    resp.raise_for_status()
    return resp


class SearchResult:
    def __init__(self, url, title, description):
        self.url = url
        self.title = title
        self.description = description

    def __repr__(self):
        return f"SearchResult(url={self.url}, title={self.title}, description={self.description})"


def search(term, num_results=10, lang="en", proxy=None, advanced=False, sleep_interval=0, timeout=5, safe="active", ssl_verify=None, region=None, start_num=0, unique=False):
    """Search the Google search engine"""

    # Proxy setup
    proxies = {"https": proxy, "http": proxy} if proxy and (proxy.startswith("https") or proxy.startswith("http") or proxy.startswith("socks5")) else None

    start = start_num
    fetched_results = 0  # Keep track of the total fetched results
    fetched_links = set() # to keep track of links that are already seen previously

    while fetched_results < num_results:
        # Send request
        resp = _req(term, num_results - start,
                    lang, start, proxies, timeout, safe, ssl_verify, region)

        # put in file - comment for debugging purpose
        # with open('google.html', 'w') as f:
        #     f.write(resp.text)

        # Parse
        soup = BeautifulSoup(resp.text, "html.parser")
        result_block = soup.find_all("div", class_="ezO2md")
        new_results = 0  # Keep track of new results in this iteration

        for result in result_block:
            # Find the link tag within the result block
            link_tag = result.find("a", href=True)
            # Find the title tag within the link tag
            title_tag = link_tag.find("span", class_="CVA68e") if link_tag else None
            # Find the description tag within the result block
            description_tag = result.find("span", class_="FrIlee")

            # Check if all necessary tags are found
            if link_tag and title_tag and description_tag:
                # Extract and decode the link URL
                link = unquote(link_tag["href"].split("&")[0].replace("/url?q=", "")) if link_tag else ""
            # Extract and decode the link URL
            link = unquote(link_tag["href"].split("&")[0].replace("/url?q=", "")) if link_tag else ""
            # Check if the link has already been fetched and if unique results are required
            if link in fetched_links and unique:
                continue  # Skip this result if the link is not unique
            # Add the link to the set of fetched links
            fetched_links.add(link)
            # Extract the title text
            title = title_tag.text if title_tag else ""
            # Extract the description text
            description = description_tag.text if description_tag else ""
            # Increment the count of fetched results
            fetched_results += 1
            # Increment the count of new results in this iteration
            new_results += 1
            # Yield the result based on the advanced flag
            if advanced:
                yield SearchResult(link, title, description)  # Yield a SearchResult object
            else:
                yield link  # Yield only the link

            if fetched_results >= num_results:
                break  # Stop if we have fetched the desired number of results

        if new_results == 0:
            #If you want to have printed to your screen that the desired amount of queries can not been fulfilled, uncomment the line below:
            #print(f"Only {fetched_results} results found for query requiring {num_results} results. Moving on to the next query.")
            break  # Break the loop if no new results were found in this iteration

        start += 10  # Prepare for the next set of results
        sleep(sleep_interval)

# Prepare information per Sponsor

For each combination pair, google search the query through googlesearch-python \
Reference: https://pypi.org/project/googlesearch-python/

In [14]:
# Helper function to check NAN
def is_nan(value):
  """Check if the given value is NaN."""
  return isinstance(value, float) and math.isnan(value)

In [15]:
# Let's create two dictionaries
# Dicionary 1, Key: company name, Value: its home url
Sponsor_urls = {}
# Dicionary 2, Key: company name, Value: its possible short name
Sponsor_shortnames = {}
# Dictionary 3, Key: company name, Value: its domain
Sponsor_domains = {}

In [16]:
# Iterate through each row in the DataFrame
for idx, row in df_target.iterrows():
  company_name = row['Sponsors']
  Sponsor_urls[company_name] = row['Sponsor_URL']
  Sponsor_shortnames[company_name] = row['Sponsor_Shortname']
  Sponsor_domains[company_name] = row['Sponsor_Domain']

# Google Search the Combinations and Determine if Both Partners in one Combination Appear on the Same Page

Issue:
* (Unsolved) Google autofilling "suggested" searching results
* (Solved) Google's 429 Client Error: giving too many request at a certain timestance. Unfortunately we don't know the exact timestance and request rate, can only be temporarily prevented or delayed by adding sleep().
  * Cannot be solved using rolling proxy
  * Solved via rotate-ip library with modified googlesearch library
* (Solved) Amazon Web API ReadTimeout Error

In [ ]:
# Create gateway object and initialise in AWS
gateway = ApiGateway("https://www.google.com",
                     access_key_id="",
                     access_key_secret="")
gateway.start()

# Assign gateway to session
session = requests.Session()
session.mount("https://www.google.com", gateway)

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://www.google.com - IP Rotate API' (10 new).


In [ ]:
target = ""

In [ ]:
# Initialize an empty list to store search results
search_results = []
retries = 10

# Loop through all combinations and track progress
for index, combination in tqdm(enumerate(Sponsor_combinations, 1), total=len(Sponsor_combinations), desc="Processing combinations"):
  # Add a try and catch
  for attempt in range(retries):
    try:
      # Create the search query by joining the two sponsors
      # We want to use exact match, that both company name (exact name) must be appearing in the search results
      query = f"\"{combination[0]}\" \"{combination[1]}\""
      # Perform a Google search and fetch the top 5 URLs
      # We will use advanced search to simulate actual google search
      top_urls = list(search(query, num_results=10, advanced=True))
      break  # If successful, break out of the retry loop
    # if 429 Error happens, quickly catch and restart as we are rolling IPs ;)
    # Same for ReadTimeout Error from Amazon Web API
    except HTTPError as http_err:
      if http_err.response.status_code == 429:
        print("429 Too Many Requests detected, will restart soon")
      else:
        raise http_err
    except ReadTimeout:
        print(f"ReadTimeout detected on attempt {attempt + 1}, retrying...")

  # Initialize variables
  Predicted_label = -1
  Representing_result = None
  # Next we will do a relationship prediction

  # -1: Something wrong with the searching process

  # 0.5: These two companys could potentially be partner, we are not sure
  # Happens when the relationship is not 0, 0.1, or 1 (has result showing both company name appear in a non-target website)
  # Also happens when Google automatically "suggesting" results, to be safe, they are assigned here instead of 0
  # Representing_Url is not speicifcally listed, thus None

  # 0: These two companys are not related
  # Happens when there is no results in the returning top_urls
  # Or the returning top_urls only contain contents that related to original target
  # Representing_Url is not speicifcally listed, thus None

  # 1: These two companys are very likely related
  # When there is 1 returning results (website) belongs to one of the two companys
  # Meaning, one company's name is mentioned by the other company
  # We can check this by checking if the target url is an extension of either company's main webpage
  # And the other company's name is mentioned in the description
  # Or if the title of the article mentions the name/shortname of both company
  # Representing_Url will be that company related result, end further iteration
  # Update: Remove third party (due to volume)

  # 0.1: These two companys are very likely not related
  # This label only appears when the resulting url is exactly the url of one company in the pair
  # But the other company's name/shortname is not mentioned in the description
  # And there is no other results as well
  # This is likely due to google's auto "suggesting" result
  # Representing_Url will be that suspicious target

  # Code to fullfill the purpose
  # No results returned
  if len(top_urls) == 0:
    Predicted_label = 0
  else:
    # Retrieve company's url, domain and shortname through dictionary
    if not is_nan(Sponsor_urls[combination[0]]):
      url_0 = Sponsor_urls[combination[0]]
    else:
      url_0 = "None"
    if not is_nan(Sponsor_domains[combination[0]]):
      domain_0 = Sponsor_domains[combination[0]]
    else:
      domain_0 = "None"
    if not is_nan(Sponsor_shortnames[combination[0]]):
      shortname_0 = Sponsor_shortnames[combination[0]].lower()
    else:
      shortname_0 = "None"
    if not is_nan(Sponsor_urls[combination[1]]):
      url_1 = Sponsor_urls[combination[1]]
    else:
      url_1 = "None"
    if not is_nan(Sponsor_domains[combination[1]]):
      domain_1 = Sponsor_domains[combination[1]]
    else:
      domain_1 = "None"
    if not is_nan(Sponsor_shortnames[combination[1]]):
      shortname_1 = Sponsor_shortnames[combination[1]].lower()
    else:
      shortname_1 = "None"


    # Check each returning result
    for result in top_urls:
      url = result.url
      title = result.title.lower()
      description = result.description.lower()

      # utilize regular expression to ensure matched word is isolated
      # \b: Word boundary ensures no characters before or after the word.
      # [\s.,!?]?: Optional space or punctuation after the shortname
      # r is needed to ensure regex work as entended, ex. r'\b' to ensure \b as word boundary instead of simply b
      pattern_name_0 = r'\b' + re.escape(combination[0].lower()) + r'[\s.,!?]?\b'
      pattern_name_1 = r'\b' + re.escape(combination[1].lower()) + r'[\s.,!?]?\b'
      pattern_shortname_0 = r'\b' + re.escape(shortname_0) + r'[\s.,!?]?\b'
      pattern_shortname_1 = r'\b' + re.escape(shortname_1) + r'[\s.,!?]?\b'

      # We do not want target to be involved
      # This does not help us at all
      if target.lower() in url or target.lower() in title:
        # If Predicted_label has been assigned to 0.5, then keep it
        # Else, make it 0
        Predicted_label = max (0, Predicted_label)
      else:
        # Both company's names are mentioned in the title
        # Lowered label to 0.5 from 1
        if (re.search(pattern_name_0, title) and re.search(pattern_name_1, title)) or (re.search(pattern_shortname_0, title) and re.search(pattern_name_1, title)) or (re.search(pattern_name_0, title) and re.search(pattern_shortname_1, title)) or (re.search(pattern_shortname_0, title) and re.search(pattern_shortname_1, title)):
          Predicted_label = max (0.5, Predicted_label)
          # Representing_result = result
        # url is an extension of company 0's homepage and name of company 1 is mentioned in the description
        elif (url_0 or domain_0) in url and (re.search(pattern_name_1, description) or re.search(pattern_shortname_1, description)):
          Predicted_label = 1
          Representing_result = result
          break
        # url is an extension of company 1's homepage and name of company 0 is mentioned in the description
        elif (url_1 or domain_1) in url and (re.search(pattern_name_0, description) or re.search(pattern_shortname_0, description)):
          Predicted_label = 1
          Representing_result = result
          break
        # url is exact match of one company, but the other company's name is not mentioned at all
        # elif (url_0 == url and shortname_1 not in description and combination[1].lower() not in description) or (url_1 == url and shortname_0 not in description and combination[0].lower() not in description):
        #   old_label = Predicted_label
        #   Predicted_label = max(0.1, Predicted_label)
            # If old_label == 0.5, no update
            # If old_label == 0.1, no update
            # If old_label == 0 or -1, update
        #   if old_label < Predicted_label:
        #     Representing_result = result
        else:
          # Both companies are mentioned in a third-party website
          # Continue checking the rest urls if there are any
          # We cannot make a conclusion yet
          # Pruned due to Professor's request
          Predicted_label = max(0, Predicted_label)

  # Append the result as a dictionary
  search_results.append({
    'Sponsor 1': combination[0],
    'Sponsor 2': combination[1],
    'Top 5 Results': top_urls,
    'Top 5 Results Title': [result.title for result in top_urls],
    'Predicted Label': Predicted_label,
    'Representing Result': Representing_result
  })

  # Delay time between each search
  # delay_time = 1  # Delay 5 seconds between each sponsor
  # time.sleep(delay_time)

Processing combinations:   2%|▏         | 2/91 [00:02<01:31,  1.02s/it]

429 Too Many Requests detected, will restart soon
429 Too Many Requests detected, will restart soon


Processing combinations:   3%|▎         | 3/91 [00:08<05:23,  3.67s/it]

429 Too Many Requests detected, will restart soon
429 Too Many Requests detected, will restart soon
429 Too Many Requests detected, will restart soon
429 Too Many Requests detected, will restart soon


Processing combinations:   4%|▍         | 4/91 [00:13<06:03,  4.18s/it]

...

Processing combinations:  98%|█████████▊| 89/91 [02:38<00:02,  1.41s/it]

429 Too Many Requests detected, will restart soon
429 Too Many Requests detected, will restart soon


Processing combinations:  99%|█████████▉| 90/91 [02:40<00:01,  1.53s/it]

429 Too Many Requests detected, will restart soon


Processing combinations: 100%|██████████| 91/91 [02:42<00:00,  1.78s/it]


In [19]:
gateway.shutdown()

Deleting gateways for site 'https://www.google.com'.
Deleted 10 endpoints with for site 'https://www.google.com'.


['10skls3cse',
 'os3by3l9ld',
 't21tcx2qra',
 'tzp8sgq1gk',
 'gdrcsau0lg',
 'o77ovnjux3',
 'vqarcgp6wj',
 'w3t526j3cf',
 'f7ksckike7',
 'sue1ahyo9j']

In [20]:
# Checking method
print("Length of combinations:", len(Sponsor_combinations))
print("Length of search_results:", len(search_results))

Length of combinations: 91
Length of search_results: 91


In [21]:
# Convert the search results into a pandas DataFrame
df_results = pd.DataFrame(search_results)

In [22]:
# Set display options to show full content
pd.set_option('display.max_colwidth', None)

In [23]:
# df_results.iloc[0]["Top 5 Results Title"]

In [24]:
# extract results that are predicted to be 1 labeled
df_1_result = df_results[df_results['Predicted Label'] == 1]

# Reset the index of the filtered DataFrame and drop the old index
df_1_result = df_1_result.reset_index(drop=True)

In [25]:
df_1_result

,Sponsor 1,Sponsor 2,Top 5 Results,Top 5 Results Title,Predicted Label,Representing Result
0,Access to Capital for Entrepreneurs,JP Morgan Chase,"[SearchResult(url=https://media.chase.com/news/launch-of-new-chase-lounge-at-the-russell-center-will-help-fuel-path-to-success-for-black-business-owners-and-entrepreneurs-in-atlanta, title=Launch of New Chase Lounge at the Russell Center Will Help Fuel ..., description= Feb 25, 2021 · ... Access to Capital for Entrepreneurs and Atlanta Wealth Building Initiative. ... © 2025 JP Morgan Chase & Co. ), SearchResult(url=https://aceloans.org/news-events/page/6/?query-0-page=15, title=News & Events | Access to Capital for Entrepreneurs, description= ACE Partnership with LISC Atlanta Continues with $100 Million commitment from JP Morgan Chase to The Entrepreneurs of Color Loan Fund (EOCF). May 10, 2022. ACE ... ), SearchResult(url=https://www.kauffman.org/wp-content/uploads/2023/06/Access-to-Capital-for-Entrepreneurs-Report-2-June-2023.pdf, title=[PDF] Access to Capital for Entrepreneurs: Removing Barriers (2023), description= (2023). “Access to Capital for Entrepreneurs: Removing Barriers: 2023 ... * BANK OF AMERICA, CITIBANK, JP MORGAN CHASE, WELLS FARGO. RATIO, KC:US. 1.6. ), SearchResult(url=https://www.ascendatl.org/, title=Ascend Atlanta – Growing minority owned businesses, description= JP Morgan Chase & Co. has partnered with Morehouse Innovation & Entrepreneurship Center, Village Micro Fund, Access to Capital for Entrepreneurs, and ... ), SearchResult(url=https://media.chase.com/news/jpmc-to-add-25-branches-in-georgia, title=JPMorgan Chase to Add 25 Branches in Georgia in Next Three Years, description= Sep 25, 2018 · ... Access to Capital for Entrepreneurs Inc., Atlanta Neighborhood ... © 2025 JP Morgan Chase & Co. ), SearchResult(url=https://finance.yahoo.com/news/c-k-mcwhorter-cancels-j-173800912.html, title=C.K. McWhorter Cancels J.P. Morgan Communications, Citing Lack ..., description= Mar 26, 2024 · ... discussions with JP Morgan Chase & Co. The decision stems from a stark observation of the bank's inade. ), SearchResult(url=https://fox2now.com/business/press-releases/ein-presswire/659210053/ace-access-to-capital-for-entrepreneurs-receives-largest-philanthropic-donation-ever-from-cooperative-assistance-fund, title=ACE | Access to Capital for Entrepreneurs Receives Largest ... - FOX 2, description= Oct 3, 2023 · ACE | Access to Capital for Entrepreneurs Receives Largest Philanthropic Donation Ever From Cooperative Assistance Fund ... JP Morgan Chase & ... ), SearchResult(url=https://aceloans.org/ace-announces-award-categories-for-its-2024-annual-awards-fundraiser/, title=ACE Announces Award Categories for Its 2024 Annual Awards ..., description= ACE | Access to Capital for Entrepreneurs Inc. is pleased to announce the ... Blank Family Foundation, JP Morgan Chase & Company, and PNC Bank Access ... ), SearchResult(url=https://fox5sandiego.com/business/press-releases/ein-presswire/746868347/ace-annual-awards-highlights-its-24-year-legacy-of-growing-georgia-businesses, title=ACE Annual Awards Highlights Its 24-Year Legacy of Growing ..., description= ... Access to Capital for Entrepreneurs held its ... JP Morgan Chase & Company, PNC Bank, Wells Fargo ... Access to Capital for Entrepreneurs Inc. +1 470 ... ), SearchResult(url=https://news.morehouse.edu/inside-morehouse/morehouse-collegeentrepreneurship-center-shares-in-640000-donationfrom-jpmorgan-chase-company, title=Morehouse College Entrepreneurship Center Shares In $640000 ..., description= Jun 13, 2019 · Access to Capital for Entrepreneurs will provide small loans to ... Including donations to other Ascend Atlanta partners, JP Morgan Chase ... )]","[Launch of New Chase Lounge at the Russell Center Will Help Fuel ..., News & Events | Access to Capital for Entrepreneurs, [PDF] Access to Capital for Entrepreneurs: Removing Barriers (2023), Ascend Atlanta – Growing minority owned businesses, JPMorgan Chase to Add 25 Branches in Georg

In [26]:
# Manually view predicted-to-be-related results
# df_1_result.iloc[0]

In [27]:
# Export to csv file
df_results.to_csv('search_results_web_search.csv', index=False)
df_1_result.to_csv('search_results_web_search_1.csv', index=False)

In [28]:
# Export to Excel file
# df_results.to_excel('search_results_web_search.xlsx', index=False)
# df_1_result.to_excel('search_results_web_search_1.xlsx', index=False)

In [29]:
# Reset the option back to default if needed
pd.reset_option('display.max_colwidth')